In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["PINECONE_API_KEY"] = os.environ['PINECONE_API_KEY']

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 6


In [16]:
# TestSet Generation
import pandas as pd

# questions and answers pairs (Created using the data from pdf and GPT-4o model)
# The questions are varied in the given varieties, distributions={simple: 50%, reasoning: 25%, multi_context: 25%}
data_varied = {
    "Question": [
        "What is included in the policy documentation?",
        "What steps should you take to make a claim on your car insurance?",
        "What does Section 1: Liability cover in the car insurance policy?",
        "What protection does Section 2: Fire and Theft provide?",
        "What are the benefits of Section 3: Courtesy Car?",
        "How does Section 4: Accidental Damage protect you?",
        "What is covered under Section 5: Windscreen Damage?",
        "What personal benefits are included in Section 6?",
        "What does Section 7: Motor Legal Cover provide?",
        "How does Section 8: Guaranteed Hire Car Plus enhance your coverage?",
        "What is the benefit of Section 9: Protected No Claim Discount?",
        "What driving locations are covered under this insurance policy?",
        "What types of losses are not covered by this insurance policy?",
        "What should you do if you have a complaint about the insurance policy?",
        "What is the purpose of the 'Glossary' in the policy document?",
        "How does the policy handle modifications to your car?",
        "What does the term 'Comprehensive with DriveSure' mean?",
        "What should you do immediately after an accident?",
        "What are the benefits of having DriveSure included in your policy?",
        "How can you contact customer service for assistance?",
        "What is the procedure for adding a new driver to your policy?",
        "How does the policy define 'your car'?",
        "What should you check in your policy details document?",
        "How can you cancel your car insurance policy?",
        "What is the impact of claims on your no-claims discount?",
        "How often should you review your car insurance policy?",
        "What factors affect your car insurance premium?",
        "How can you get a copy of your insurance certificate?",
        "What additional covers are available under this policy?",
        "What is the role of Green Flag breakdown cover in the policy?"
    ],
    "Answer": [
        "The policy documentation includes this booklet, your car insurance details, and your certificate(s) of motor insurance. If the policy includes Green Flag breakdown cover, it also includes the Green Flag policy booklet. If the policy includes DriveSure, it includes the DriveSure terms and conditions.",
        "To make a claim, ensure your safety, contact the insurance provider, report the incident, provide necessary details, and follow the insurer's instructions.",
        "Section 1: Liability covers legal liability for injury to others and damage to their property resulting from an accident involving your car.",
        "Section 2: Fire and Theft covers loss or damage to your car caused by fire, theft, or attempted theft, including the cost of repairs or replacement.",
        "Section 3: Courtesy Car provides a temporary replacement car if your vehicle is being repaired following an insured accident.",
        "Section 4: Accidental Damage covers the cost of repairing or replacing your car if it is damaged in an accident.",
        "Section 5: Windscreen Damage covers the repair or replacement of your car's windscreen and windows if they are damaged.",
        "Section 6: Personal Benefits covers various benefits, including personal accident coverage for you and your passengers, medical expenses incurred following an accident, and personal belongings damaged or stolen from your car.",
        "Section 7: Motor Legal Cover provides legal assistance and cover for legal costs incurred in pursuing a claim for uninsured losses following a motor accident.",
        "Section 8: Guaranteed Hire Car Plus provides you with a hire car if your vehicle is stolen or written off.",
        "Section 9: Protected No Claim Discount allows you to make a certain number of claims without affecting your no-claims discount.",
        "The policy covers driving within the geographical limits specified in the policy booklet. This typically includes driving within the UK and may extend to certain European countries, subject to the terms and conditions outlined in the policy.",
        "The policy does not cover losses such as wear and tear, mechanical or electrical breakdown, damage caused by using the car for unlawful purposes, and any loss or damage if the driver does not have a valid driving license. Detailed exclusions are listed in the 'Losses we don't cover' section.",
        "If you have a complaint, you should contact the insurance provider's customer service department. Provide details of your complaint and any relevant information. The insurer will investigate and respond to your complaint in accordance with their complaints handling process, aiming to resolve it satisfactorily.",
        "The 'Glossary' section provides definitions for specific terms and phrases used throughout the policy document. This helps you understand the exact meanings of these terms, ensuring clarity and avoiding misunderstandings.",
        "Modifications to your car should be reported to the insurer. The policy may not cover damage or incidents resulting from unreported modifications. Always check your policy for specific terms regarding modifications.",
        "The term 'Comprehensive with DriveSure' means that your policy includes all the benefits of a Comprehensive policy, along with additional features and services provided under the DriveSure program.",
        "Immediately after an accident, ensure everyone's safety and move to a safe location if possible. Exchange contact and insurance details with other parties involved, and contact your insurer to report the incident.",
        "DriveSure provides additional benefits such as regular vehicle health checks, driving tips, and possibly lower premiums based on safe driving habits. It enhances the standard coverage of a Comprehensive policy.",
        "You can contact customer service for assistance through the phone number or email address provided in your policy documents. Customer service can help with policy inquiries, claims, and other support.",
        "To add a new driver to your policy, contact your insurance provider with the details of the new driver, including their driving history and personal information. The insurer will update your policy and adjust your premium if necessary.",
        "The policy defines 'your car' as the vehicle described in your certificate of motor insurance, including its standard accessories and any modifications declared to and accepted by the insurer.",
        "In your policy details document, check for accurate information about your vehicle, coverage limits, premium amounts, and any endorsements or exclusions. Ensure all information is correct and up to date.",
        "To cancel your car insurance policy, contact your insurer directly. Provide your policy number and the date you want the cancellation to take effect. Be aware of any cancellation fees or refund policies.",
        "Claims can affect your no-claims discount by reducing the number of consecutive claim-free years. However, if you have a Protected No Claim Discount, you can make a certain number of claims without losing the discount.",
        "You should review your car insurance policy annually or whenever you experience significant changes, such as purchasing a new car, moving to a new location, or adding a new driver to your policy.",
        "Factors affecting your car insurance premium include your age, driving history, the make and model of your car, where you live, how you use your car, and any additional coverage or features you choose.",
        "You can get a copy of your insurance certificate by contacting your insurer or accessing your account on the insurer's website. A digital or physical copy will be provided upon request.",
        "Additional covers available under this policy may include breakdown cover, legal protection, personal accident coverage, and windscreen cover. These options provide extra protection beyond standard coverage.",
        "Green Flag breakdown cover provides roadside assistance and recovery services if your car breaks down. It includes services such as towing, emergency repairs, and assistance with flat tires or battery issues."
    ]
}

df_varied = pd.DataFrame(data_varied)
df_varied.to_csv('data/testset.csv', index=False)

In [3]:
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_groq import ChatGroq
from pinecone import Pinecone
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)
from ragas import evaluate

load_dotenv()

os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')
os.environ["GROQ_API_KEY"] = os.environ['GROQ_API_KEY']

embeddings = OllamaEmbeddings(model="all-minilm:latest")
pc = Pinecone()
index_name = "rag-pdf"
index = pc.Index(index_name)


Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 6


In [4]:
def retriever(input):
    input_em = embeddings.embed_query(input)
    result = index.query(vector=input_em, top_k=2, includeMetadata=True)
    texts = []
    for match in result["matches"]:
        if "metadata" in match and "text" in match["metadata"]:
            texts.append(match["metadata"]["text"])
    return " ".join(texts)

In [5]:
llm = ChatGroq(model = "mixtral-8x7b-32768")

buffer_memory = ConversationBufferWindowMemory(k=3, return_messages=True)

system_msg_template = SystemMessagePromptTemplate.from_template(template="""Answer the question as truthfully as possible using the provided context, 
and if the answer is not contained within the text below, say 'I don't know'""")

human_msg_template = HumanMessagePromptTemplate.from_template(template="{input}")

prompt_template = ChatPromptTemplate.from_messages([system_msg_template, MessagesPlaceholder(variable_name="history"), human_msg_template])

conversation = ConversationChain(memory=buffer_memory, prompt=prompt_template, llm=llm, verbose=True)

In [9]:
import pandas as pd

df = pd.read_csv('data/testset.csv')
questions = df["question"].to_list()
ground_truth = df["ground_truth"].to_list()

In [14]:
import json
import os
from time import sleep
from datasets import Dataset

def save_intermediate_results(data, filename="intermediate_results.json"):
    with open(filename, 'w') as f:
        json.dump(data, f)

def load_intermediate_results(filename="intermediate_results.json"):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

filename = "intermediate_results.json"
data = load_intermediate_results(filename)
batch_size = 2
pause_time = 300

# Determine the starting point in case of a previous interruption
start_index = len(data["question"])

for i in range(start_index, len(questions), batch_size):
    batch_questions = questions[i:i+batch_size]
    batch_contexts = []

    for query in batch_questions:
        context = retriever(query)
        data["question"].append(query)
        data["contexts"].append(context)
        batch_contexts.append(context)

    for query, context in zip(batch_questions, batch_contexts):
        data["answer"].append(conversation.predict(input=f"Context:\n{context}\n\nQuery:\n{query}"))

    save_intermediate_results(data, filename)
    
    if i + batch_size < len(questions):
        sleep(pause_time)

dataset = Dataset.from_dict(data)




> Entering new ConversationChain chain...
Prompt after formatting:
System: Answer the question as truthfully as possible using the provided context, 
and if the answer is not contained within the text below, say 'I don't know'
Human: Context:
of motor insurance.
If the policy includes Green Flag breakdown cover:
 >Your breakdown cover and your car 
insurance are part of the same policy.
 >The policy also includes the Green Flag 
policy booklet we’ve given you.
If you have a policy that includes DriveSure:
 >The policy also includes the DriveSure terms 
and conditions we’ve given you.
Please read all these documents carefully  and keep them safe in case you need them.Words in bold type
Some of the words and phrases we use in 
this booklet have a specific meaning – for example, your car or modifications.
We’ve highlighted these words using bold type. You can find the exact meanings of  these words in the ‘Glossary’ on page 4,  or at the start of each section.
Comprehensive with DriveSu

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01hzsmtdtkfygt5jc4gvfnmy0e` on tokens per minute (TPM): Limit 5000, Used 0, Requested ~33684. Please try again in 5m44.208s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}